Auto-Clean-Test

In [1]:
import logging
import os
import shutil
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean

/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digennoise = pickle.load(f)

#print(digennoise.keys())

with open("./data/DigenData/all_new_test_sets_no_noise.pkl",'rb') as f:
    digen = pickle.load(f)

#print(digen.keys())
#print(digen['digen8_4426'])
#Test= pd.DataFrame(digennoise['digen8_4426']['X'])
#yval = pd.DataFrame(digennoise['digen8_4426']['y'])
#current_noise = digennoise['digen8_4426']['noise']
#yval.columns = ['y']
#yval.rename(columns = {'0':'y'}, inplace = True)
#out = pd.concat([Test, yval], axis=1)
#out.index.name = 'InstanceId'
#out.to_csv('./data/DigenStudy/digen8_4426.csv')
#print(out)

In [3]:

data = pd.DataFrame(digennoise['digen8_4426']['X'])
yval = pd.DataFrame(digennoise['digen8_4426']['y'])
current_noise = digennoise['digen8_4426']['noise']
yval.columns = ['y']
yval.rename(columns = {'0':'y'}, inplace = True)
out = pd.concat([data, yval], axis=1)
out.index.name = 'InstanceId'
digennoise['digen8_4426'] = current_noise
if current_noise == 0.0: 
    outstring = './data/DigenStudy/No_Noise/' + 'digen8_4426' + '.csv'
    #tempstring = './data/testfolder/' + 'digen8_4426' + '.csv'
    out.to_csv(outstring)
    #out.to_csv(tempstring)
else:
    outstring = './data/DigenStudy/Noisy/' + 'digen8_4426' + '.csv'
    #tempstring = './data/testfolder/' + 'digen8_4426' + '.csv'
    out.to_csv(outstring)
    #out.to_csv(tempstring)

X_train, X_test,y_train, y_test = train_test_split(data, yval,test_size=0.08, shuffle=True)

train_set = pd.concat([X_train, y_train], axis =1)
train_set.index.name = 'InstanceId'
tempstring = './data/testfolder/' + 'digen8_4426' +'_train.csv'
train_set.to_csv(tempstring)
cleanrun = OptimizeClean(dataset_name=['digen8_4426_train'], n_trials=1, data_path='./data/testfolder', class_label='y', instance_label="InstanceId")
cleanrun.run(run_para=True)
optimal = cleanrun.param
current_auc = cleanrun.goal
os.remove(tempstring)


[I 2023-07-05 11:38:41,076] A new study created in memory with name: no-name-94ffd348-9bc4-432c-af40-222b258e3cf1
Best trial: 0. Best value: 0.977162:  10%|█         | 1/10 [02:52<25:54, 172.74s/it]

digen8_4426_train digen8_4426_train


Best trial: 1. Best value: 0.98698:  20%|██        | 2/10 [07:30<31:15, 234.41s/it] 

digen8_4426_train digen8_4426_train


Best trial: 1. Best value: 0.98698:  30%|███       | 3/10 [09:25<20:59, 179.93s/it]

...

Best trial: 1. Best value: 0.98698:  90%|█████████ | 9/10 [29:06<03:28, 208.11s/it]

digen8_4426_train digen8_4426_train


Best trial: 1. Best value: 0.98698: 100%|██████████| 10/10 [31:23<00:00, 188.31s/it]

digen8_4426_train digen8_4426_train


<Figure size 600x600 with 0 Axes>

categorical cut = 5
sig cutoff = 0.0198
feature missing = 0.7
cleaning miss = 0.6
correlation removal = 0.9
partition method: random
n_splits = 9

424 min runtime for 1 dataset with 100 trials. 
= 7 hours.

In [4]:
print(optimal)

{'categorical_cutoff': 9, 'sig_cutoff': 0.007051812409598334, 'featureeng_missingness': 0.8, 'cleaning_missingness': 0.15000000000000002, 'correlation_removal_threshold': 0.95, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Random', 'n_splits': 4}


In [5]:
tempstring = './data/testfolder/' + 'digen8_4426' +'_test.csv'
test_set = pd.concat([X_test, y_test], axis =1)
test_set.index.name = 'InstanceId'
test_set.to_csv(tempstring)

most_recent_run = AutoRunner(**optimal, dataset_names=['digen8_4426_test'],data_path='./data/testfolder',
                                    gen_report=False, clean=False, class_label='y', 
                                    instance_label='InstanceId', ml_algorithms=["NB", "LR", "DT", "EN", "XGB", "LGB", "CGB", "SVM","GB", "RF"], 
                                    exclude=["ANN","KNN","GP", 'eLCS', 'XCS', "ExSTraCS"]) # "XGB", "LGB", "CGB", "SVM","GB", "RF"
output_csv = most_recent_run.run(run_para=True)
performance = pd.read_csv(output_csv)
summary_chart = performance
goal = performance["ROC AUC"].max()
best_model = performance.loc[performance["ROC AUC"].idxmax()][0]
png_out = output_csv.removesuffix('Summary_performance_mean.csv')
png_out = png_out + 'Summary_ROC.png'
final_model_comparison = plt.show(png_out)
print(current_auc-goal)
os.remove(tempstring)

100%|██████████| 16/16 [00:00<00:00, 17349.76it/s]


metadata.pickle digen8_4426_test
digen8_4426_test digen8_4426_test
0.3584103603840939


In [6]:
print(cleanrun.param)

{'categorical_cutoff': 9, 'sig_cutoff': 0.007051812409598334, 'featureeng_missingness': 0.8, 'cleaning_missingness': 0.15000000000000002, 'correlation_removal_threshold': 0.95, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Random', 'n_splits': 4}


trials: 1 difference: 0.34 param: {'categorical_cutoff': 10, 'sig_cutoff': 0.02896993117424986, 'featureeng_missingness': 0.3, 'cleaning_missingness': 0.8, 'correlation_removal_threshold': 0.9, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Random', 'n_splits': 9}

trials: 10 difference: 0.35 param: {'categorical_cutoff': 9, 'sig_cutoff': 0.007051812409598334, 'featureeng_missingness': 0.8, 'cleaning_missingness': 0.15000000000000002, 'correlation_removal_threshold': 0.95, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Random', 'n_splits': 4}

In [7]:
# autotest = AutoRunner(dataset_names=['digen8_4426'], clean=False, class_label='y', instance_label='InstanceId')

In [8]:
#output_csv = autotest.run()

In [9]:
#print(output_csv)

#performance = pd.read_csv(output_csv)
#print(performance)
#print(performance['ROC AUC'].max())
#print(performance.loc[performance['ROC AUC'].idxmax()][0])